In [1]:
import sympy
from sympy import Rational
import numpy as np
import pandas as pd
import math
from gurobipy import *

train_info = pd.read_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_multiclass\train_info_multi_class_multi_origin0612.csv')
commuter_data = pd.read_csv(r'D:/学习/课题/inverse optimization data/multi_class_multi_origin_commuter_data.csv')

#变量维度:车次，站间区间
di = len(train_info['TYPE'])
station_interval = list(range(151019789, 151019778, -1))

#灵敏度分析邻域半径
s = 0.0002

alpha_value_initial = 0.000354 #0.2#0.1282 
beta_value_initial = 0.002325 #0.2#0.1476 
gamma_value_initial = 0.002251 #0.2#0.1545 
g0_value_initial = 0.000029 #0.2#0.0042 

#上层决策变量值记录
upper_parameters_record = sympy.Matrix([alpha_value_initial, beta_value_initial, gamma_value_initial, g0_value_initial])
#上层目标值记录
upper_obj_record = []
#收敛准侧值记录
indicator_record = []

origins_input = [151019789, 151019787, 151019785, 151019784]
origins_interval_num = {151019789:11, 151019787:9, 151019785:7, 151019784:6}
userclass_input = {origins_input[0]:[0, 1, 2, 3, 4], origins_input[1]:[0, 1, 2, 3, 4, 5], origins_input[2]:[0, 1, 2, 3, 4], origins_input[3]:[0, 1, 2, 3, 4, 5]}

train_choice_set_input = {}
train_choice_set_input[origins_input[0]] = {0:list(range(14,32)), 1:list(range(21,47)), 2:list(range(19,42)), 3:list(range(26,47)), 4:list(range(8,32))}
train_choice_set_input[origins_input[1]] = {0:list(range(24,42)), 1:list(range(14,32)), 2:list(range(16,35)), 3:list(range(26,47)), 4:list(range(5,27)), 5:list(range(24,45))}
train_choice_set_input[origins_input[2]] = {0:list(range(29,47)), 1:list(range(4,27)), 2:list(range(14,32)), 3:list(range(16,35)), 4:list(range(19,45))}
train_choice_set_input[origins_input[3]] = {0:list(range(11,30)), 1:list(range(19,35)), 2:list(range(24,42)), 3:list(range(16,30)), 4:list(range(26,47)), 5:list(range(3,25))}

demand_input = {}
demand_input[origins_input[0]] = {0: 155, 1: 81, 2: 112, 3: 53, 4: 95}
demand_input[origins_input[1]] = {0: 60, 1: 134, 2: 100, 3: 71, 4: 109, 5: 56}
demand_input[origins_input[2]] = {0: 57, 1: 91, 2: 100, 3: 87, 4: 95}
demand_input[origins_input[3]] = {0: 74, 1: 125, 2: 113, 3: 98, 4: 81, 5: 60}

desired_arrival_time_input = {}
desired_arrival_time_input[origins_input[0]] = {0:30900, 1:33300, 2:32100, 3:34500, 4:29700}
desired_arrival_time_input[origins_input[1]] = {0:32700, 1:30900, 2:32100, 3:34500, 4:29700, 5:33300}
desired_arrival_time_input[origins_input[2]] = {0:34500, 1:29700, 2:30900, 3:31500, 4:32700}
desired_arrival_time_input[origins_input[3]] = {0:30300, 1:31500, 2:32700, 3:30900, 4:34500, 5:29100}

#出行时间
T = {}
for o in origins_input:
    for j in train_info['TYPE']:
        T[(o,j)] = train_info[train_info['TYPE'] == j]['TRIP_TIME_average_%d'%o].iloc[0]
        
#乘车时间
Tv = {151019789: 110, 151019788: 130, 151019787: 140, 151019786: 120, 151019785: 160, 151019784: 180, 151019783: 180, 151019782: 180, 151019781: 200, 151019780: 170, 151019779: 110}

#n0既有流量
n0 = {}
for j in train_info['TYPE']:
    for i in station_interval:
        n0[(j,i)] = train_info[train_info['TYPE'] == j]['%d'%i].values[0]

#TE和TL
TE = {}
TL = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            TE[(o,i,j)] = max(desired_arrival_time_input[o][i] - train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0], 0)
            TL[(o,i,j)] = max(train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0] - desired_arrival_time_input[o][i], 0)

In [2]:
#定义符号
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
g0 = sympy.Symbol('g0')

lamb = sympy.Symbol('lamb')
no_num = sympy.Symbol('no_num')

nis = {}
nis_keys = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis[(o,i,j)] = sympy.Symbol('n%d,%d,%d'%(o,i,j))
            nis_keys.append((o,i,j))
miu = {}
for o in origins_input:
    for i in userclass_input[o]:
        miu[(o,i)] = sympy.Symbol('u%d,%d'%(o,i))

y_observed_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            y_observed_li.append(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)][j-1])
y_observed = sympy.Matrix(y_observed_li)        
        
y_observed_dic = {}
for o in origins_input:
    y_observed_dic[o] = {}
    for i in userclass_input[o]:
        y_observed_dic[o][i] = sympy.Matrix(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)])

upper_parameters_value = {}
upper_parameters_value[alpha] = alpha_value_initial
upper_parameters_value[beta] = beta_value_initial
upper_parameters_value[gamma] = gamma_value_initial
upper_parameters_value[g0] = g0_value_initial

In [3]:
flow_nis_aggregate = sympy.zeros(di,len(station_interval))
flow_nis_aggregate_dic = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] = flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(nis[(o,i,j)], (1, origins_interval_num[o])))

for j in train_info['TYPE']:
    for i in station_interval:
        flow_nis_aggregate_dic[(j, i)] = flow_nis_aggregate[(j - 1),(151019789 - i)]  

#车厢内拥挤成本
def invehicle_crowding_cost(origin, train, Tv_dic, n0_dic, flow_dic, g0_para):
    crowding_cost = 0
    for i in range(origin, 151019778, -1):
        crowding_cost = crowding_cost + g0_para * Tv_dic[i] * (flow_dic[(train, i)] + n0_dic[(train, i)] - 256) / 275
    return crowding_cost

In [4]:
#lagrange函数对n的导数
Ln = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            Ln.append((sympy.log(nis[(o,i,j)]) + 1) + invehicle_crowding_cost(o, j, Tv, n0, flow_nis_aggregate_dic, g0) + alpha * T[(o,j)] + beta * TE[(o,i,j)] + gamma * TL[(o,i,j)] - miu[(o,i)])            
            
Noi = []
for o in origins_input:
    for i in userclass_input[o]:
        noi = 0
        for j in train_choice_set_input[o][i]:
            noi = noi + nis[(o,i,j)]
        Noi.append(noi) 

#分块矩阵A
A_li = []
for num in range(0,len(Ln)):
    ln = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                ln.append(sympy.diff(Ln[num],nis[(o,i,j)]))
    A_li.append(ln)
    
A = sympy.Matrix(A_li)

#分块矩阵B
B_li = []
for num in range(0,len(Ln)):
    lu = []
    for o in origins_input:
        for i in userclass_input[o]:
            lu.append(sympy.diff(Ln[num],miu[(o,i)]))
    B_li.append(lu)
    
B = sympy.Matrix(B_li)

#分块矩阵C
C_li = []
for num in range(0,len(Noi)):
    nn = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nn.append(sympy.diff(Noi[num],nis[(o,i,j)]))
    C_li.append(nn)  
    
C = sympy.Matrix(C_li)

#分块矩阵D
D_li = []
for num in range(0,len(Noi)):
    nu = []
    for o in origins_input:
        for i in userclass_input[o]:
            nu.append(sympy.diff(Noi[num],miu[(o,i)]))
    D_li.append(nu)        

D = sympy.Matrix(D_li) 

#分块矩阵N
N_li = []
for num in range(0,len(Ln)):
    lp = []
    lp.append(sympy.diff(Ln[num],alpha))
    lp.append(sympy.diff(Ln[num],beta))
    lp.append(sympy.diff(Ln[num],gamma))
    lp.append(sympy.diff(Ln[num],g0))
    N_li.append(lp)
    
N = sympy.Matrix(N_li)

epoch = 0
indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))

In [5]:
#cost_fuction
def train_cost(flow_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_dic = {}
    for o in origins_input:
        for i in userclass[o]:
            for j in train_choice_set[o][i]:           
                train_cost_dic[(o, i, j)] = invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]

    return train_cost_dic

#logit车次流量加载
def logit(origin, user, train_cost_dic, demand, choice_set):
    
    cost_exp_li = []
    for j in train_info['TYPE']:
        if j in choice_set:
            cost_exp_li.append(sympy.exp( - train_cost_dic[(origin, user, j)]))
        else:
            cost_exp_li.append(0)
            
    flow = (sympy.Matrix(cost_exp_li) / np.sum(cost_exp_li)) * demand 
    return flow

#多类用户logit加载
def multiorigin_multiclass_logit(userclass, train_cost_dic, demand_dic, train_choice_set):
    
    flow_multiorigin_multiclass_aggregate = sympy.zeros(di,len(station_interval))
    flow_multiorigin_multiclass_dic = {}
    flow_multiorigin_multiclass_aggregate_dic = {}
    
    for o in origins_input:
        flow_multiorigin_multiclass_dic[o] = {}
        for i in userclass[o]:
            flow_multiorigin_multiclass_dic[o][i] = logit(o, i, train_cost_dic, demand_dic[o][i], train_choice_set[o][i])
            flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] = flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multiorigin_multiclass_dic[o][i], (1, origins_interval_num[o])))
    
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_multiorigin_multiclass_aggregate_dic[(j, i)] = flow_multiorigin_multiclass_aggregate[(j - 1),(151019789 - i)]
    
    return flow_multiorigin_multiclass_aggregate, flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic

#cost_fuction_stochastic
def train_cost_stochastic(flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_stochastic_dic = {}
    for o in origins_input:
        train_cost_stochastic_dic[o] = {}
        for i in userclass[o]:
            flow_origin_class_dic = dict(zip(train_info['TYPE'], flow_multiorigin_multiclass_dic[o][i]))
            train_cost_stochastic_dic[o][i] = {}
            for j in train_info['TYPE']:  #保证矩阵运算时宽度一致
                if j in train_choice_set[o][i]:
                    train_cost_stochastic_dic[o][i][j] = (sympy.log(flow_origin_class_dic[j])+ 1) + invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_multiorigin_multiclass_aggregate_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]
                else:
                    train_cost_stochastic_dic[o][i][j] = no_num
                    
    return train_cost_stochastic_dic

#二分法求步长
def binary_search(equation):
    a = 0
    b = 1
    fa = equation.evalf(subs = {lamb:a})
    fb = equation.evalf(subs = {lamb:b})
    while a <= b:
        mid = (a + b) / 2
        fm = equation.evalf(subs = {lamb:mid})
        if abs(fm) < 0.1:
            #print("方程的根为",mid)
            break
        if fa * fm < 0:
            a = a
            b = mid
            fb = equation.evalf(subs = {lamb:b})
            #print("解在[",a,",",b,"]之间")
        elif fb * fm < 0:
            a = mid
            b = b
            fa = equation.evalf(subs = {lamb:a})
            #print("解在[",a,",",b,"]之间")
            
    return mid

In [6]:
def train_flow_assignment(userclass, train_choice_set, multi_demand, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    n = 0
    indicator_am = 1
    flow_dic_initial = {}
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_dic_initial[(j, i)] = 0
    cost_dic = train_cost(flow_dic_initial, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    flow_aggregate_x_new, flow_multi_dic_x_new, flow_aggregate_dic_x_new = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
    
    while (indicator_am > 0.0001) or (n <= 20):
    
        flow_multi_dic_x = flow_multi_dic_x_new
        flow_aggregate_dic_x = flow_aggregate_dic_x_new

        #车次成本更新
        cost_dic = train_cost(flow_aggregate_dic_x, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        #logit流量加载
        flow_aggregate_y, flow_multi_dic_y, flow_aggregate_dic_y = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
        
        #辅助流量计算（计算迭代步长）
        flow_aggregate_aux = sympy.zeros(di,len(station_interval))
        flow_multi_dic_aux = {}
        flow_aggregate_dic_aux = {}
        
        for o in origins_input:
            flow_multi_dic_aux[o] = {}
            for i in userclass[o]:
                flow_multi_dic_aux[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb
                flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] = flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_aux[o][i], (1, origins_interval_num[o])))

        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_aux[(j, i)] = flow_aggregate_aux[(j - 1),(151019789 - i)]
                
        #计算迭代步长
        train_cost_stochastic_aux_dic = train_cost_stochastic(flow_multi_dic_aux, flow_aggregate_dic_aux, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        equation_lamb = 0
        for o in origins_input:
            for i in userclass[o]:
                equation_lamb = equation_lamb + ((flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]).T * sympy.Matrix(list(train_cost_stochastic_aux_dic[o][i].values())))[0,0]
        #print('start')
        lamb_value_df = binary_search(equation_lamb)
        #print(lamb_value_df)
    
        #确定新的迭代起点
        flow_multi_dic_x_new = {}
        flow_aggregate_dic_x_new = {}
        flow_aggregate_x_new = sympy.zeros(di,len(station_interval))
        for o in origins_input:
            flow_multi_dic_x_new[o] = {}
            for i in userclass[o]:
                flow_multi_dic_x_new[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb_value_df
                flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] = flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_x_new[o][i], (1, origins_interval_num[o])))
    
        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_x_new[(j, i)] = flow_aggregate_x_new[(j - 1),(151019789 - i)]
        
        #收敛性检验
        numerator = 0
        denominator = 0
        for o in origins_input:
            for i in userclass[o]:
                numerator = numerator + ((flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]).T * (flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]))[0,0]
                denominator = denominator + (sympy.ones(1,di) * flow_multi_dic_x[o][i])[0,0]
        
        indicator_am = (numerator ** 0.5) / denominator
        n = n + 1
        print(indicator_am)
        print(n)
        if (indicator_am <= 0.00001) or (n >= 20):
            break
    print(indicator_am)
    print(n)
    
    cost_dic_x_new = train_cost(flow_aggregate_dic_x_new, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    
    return flow_multi_dic_x_new, flow_aggregate_x_new, flow_aggregate_dic_x_new, cost_dic_x_new

In [7]:
flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0])

0.000107726490414604
1
5.31889275433925e-5
2
2.62633361557686e-5
3
1.29690689900706e-5
4
6.40469186126864e-6
5
6.40469186126864e-6
5


In [16]:
#到达率流量取值
nis_value_dic = {}
nis_value_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
            nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         
nis_value_matrix = sympy.Matrix(nis_value_li)

#上层目标值记录
upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
print(f'上层目标值均衡:{upper_obj_record[-1]}')

上层目标值均衡:1254.63240584805


In [8]:
arr89 = np.column_stack((flow_multi_dic[151019789][0],flow_multi_dic[151019789][1],flow_multi_dic[151019789][2],flow_multi_dic[151019789][3],flow_multi_dic[151019789][4]))

In [9]:
arr87 = np.column_stack((flow_multi_dic[151019787][0],flow_multi_dic[151019787][1],flow_multi_dic[151019787][2],flow_multi_dic[151019787][3],flow_multi_dic[151019787][4],flow_multi_dic[151019787][5]))

In [10]:
arr85 = np.column_stack((flow_multi_dic[151019785][0],flow_multi_dic[151019785][1],flow_multi_dic[151019785][2],flow_multi_dic[151019785][3],flow_multi_dic[151019785][4]))

In [11]:
arr84 = np.column_stack((flow_multi_dic[151019784][0],flow_multi_dic[151019784][1],flow_multi_dic[151019784][2],flow_multi_dic[151019784][3],flow_multi_dic[151019784][4],flow_multi_dic[151019784][5]))

In [12]:
pd.DataFrame(arr89).to_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_multiclass1221\departure_rate_mnl89_0104.csv', index = False)

In [13]:
pd.DataFrame(arr87).to_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_multiclass1221\departure_rate_mnl87_0104.csv', index = False)

In [14]:
pd.DataFrame(arr85).to_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_multiclass1221\departure_rate_mnl85_0104.csv', index = False)

In [15]:
pd.DataFrame(arr84).to_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_multiclass1221\departure_rate_mnl84_0104.csv', index = False)

In [13]:
arr89

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1.40241266696422],
       [0, 0, 0, 0, 1.78267251603782],
       [0, 0, 0, 0, 1.91089456928004],
       [0, 0, 0, 0, 2.79271958425559],
       [0, 0, 0, 0, 4.04426490548091],
       [0, 0, 0, 0, 4.71007389115799],
       [2.03018659815538, 0, 0, 0, 7.51093510641739],
       [3.04204869119128, 0, 0, 0, 11.2544562919656],
       [3.40228280417600, 0, 0, 0, 12.5871950790369],
       [6.25806165180979, 0, 0, 0, 18.0279421273274],
       [7.69843420042070, 0, 0, 0, 9.52498298296821],
       [10.8551207461179, 0, 1.21161018893716, 0, 8.61249131314447],
       [17.0419850704232, 0, 1.90216625621686, 0, 4.01401044271899],
       [22.6300061915231, 0.235897966370207, 2.52588209876077, 0,
        2.42067337080785],
       [31.6076344717603, 0.329482440875438, 3.52793535686735, 0,
        1.85322462950410],
    

In [11]:
flow_multi_dic

{151019789: {0: Matrix([
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [ 2.03018659815538],
  [ 3.04204869119128],
  [   3.402282804176],
  [ 6.25806165180979],
  [  7.6984342004207],
  [ 10.8551207461179],
  [ 17.0419850704232],
  [ 22.6300061915231],
  [ 31.6076344717603],
  [ 18.3109304952145],
  [ 11.7024181813879],
  [ 8.94467512710971],
  [ 4.84165230049246],
  [ 2.62152993129484],
  [ 1.66939026206977],
  [ 1.16526344307129],
  [0.704924468441002],
  [0.473455365340848],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [      

In [7]:
while (indicator >= 0.00015) or (epoch == 0):
    
    epoch = epoch + 1
    flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0])
    
    #到达率流量取值
    nis_value_dic = {}
    nis_value_li = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
                nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         
    nis_value_matrix = sympy.Matrix(nis_value_li)
    
    #上层目标值记录
    upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
    print(f'上层目标值均衡:{upper_obj_record[-1]}')
    
    #矩阵运算所有参数取值
    matrix_parameters_value_all = {}
    matrix_parameters_value_all.update(nis_value_dic)
    matrix_parameters_value_all.update(upper_parameters_value)
    
    #矩阵计算
    A_value = np.matrix(A.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    B_value = np.matrix(B.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    C_value = np.matrix(C.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    D_value = np.matrix(D.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    N_value = np.matrix(N.evalf(subs = matrix_parameters_value_all)).astype(np.float64)

    #梯度计算
    I11_value = A_value.I + A_value.I * B_value * (D_value - C_value * A_value.I * B_value).I * C_value * A_value.I
    Grad = - I11_value * N_value    
    
    #反应函数
    x = sympy.Matrix([alpha, beta, gamma, g0])
    x0 = sympy.Matrix([matrix_parameters_value_all[alpha], matrix_parameters_value_all[beta], matrix_parameters_value_all[gamma], matrix_parameters_value_all[g0]])
    y = nis_value_matrix + Grad * (x - x0)
    
    #转化为矩阵形式，lh为系数矩阵，rh为右端项
    lhs, rhs = sympy.linear_eq_to_matrix(y - y_observed, [alpha, beta, gamma, g0])
    #转化为numpy数组
    lh = np.array(lhs).astype(np.float64)
    rh = np.array(rhs).astype(np.float64)
    
    #上层规划模型
    upper_model = Model("Bilevel upper model")
    # 约束非凸
    upper_model.Params.NonConvex = 2
    #上层决策变量
    alpha_up = upper_model.addVar(lb = upper_parameters_value[alpha] - s, ub = upper_parameters_value[alpha] + s, vtype = GRB.CONTINUOUS, name = "alpha_up")
    beta_up = upper_model.addVar(lb = upper_parameters_value[beta] - s, ub = upper_parameters_value[beta] + s, vtype = GRB.CONTINUOUS, name = "beta_up")
    gamma_up = upper_model.addVar(lb = upper_parameters_value[gamma] - s, ub = upper_parameters_value[gamma] + s, vtype = GRB.CONTINUOUS, name = "gamma_up")
    g0_up = upper_model.addVar(lb = upper_parameters_value[g0] - s, ub = upper_parameters_value[g0] + s, vtype = GRB.CONTINUOUS, name = "g0_up")

    #上层目标函数
    upper_model.setObjective(quicksum((lh[i,0] * alpha_up + lh[i,1] * beta_up + lh[i,2] * gamma_up + lh[i,3] * g0_up - rh[i,0])**2 for i in range(lh.shape[0])) ,GRB.MINIMIZE)
    
    #约束
    upper_model.addConstr( alpha_up >= 0)
    upper_model.addConstr( beta_up >= 0)
    upper_model.addConstr( gamma_up >= 0)
    upper_model.addConstr( g0_up >= 0)
    
    upper_model.optimize()    
    
    #上层目标函数优化值
    upper_model_objvalue = upper_model.ObjVal
    #上层决策变量优化值
    upper_parameters_value = {}
    upper_parameters_value[alpha] = [v.x for v in upper_model.getVars() if v.varName == 'alpha_up'][0]
    upper_parameters_value[beta] = [v.x for v in upper_model.getVars() if v.varName == 'beta_up'][0]
    upper_parameters_value[gamma] = [v.x for v in upper_model.getVars() if v.varName == 'gamma_up'][0]
    upper_parameters_value[g0] = [v.x for v in upper_model.getVars() if v.varName == 'g0_up'][0]
    
    #计算收敛准侧的指标
    upper_parameters_record = upper_parameters_record.col_insert(upper_parameters_record.shape[1], sympy.Matrix([upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0]]))
    indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))
    indicator_record.append(indicator)
    print(f'上层目标值灵敏度:{upper_model_objvalue}')
    print(f'indicator:{indicator}')
    print(f'参数结果：{upper_parameters_value}')
    print(f'epoch:{epoch}')

0.000122376874743034
1
6.03208807258814e-5
2
2.97374866342538e-5
3
1.46625437255923e-5
4
7.23074370541017e-6
5
7.23074370541017e-6
5
上层目标值均衡:1327.95239601587
Using license file c:\gurobi\gurobi.lic
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x9f298313
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+05, 7e+06]
  QObjective range [3e+07, 4e+10]
  Bounds range     [2e-04, 2e-03]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering t

KeyboardInterrupt: 

In [11]:
np.savetxt('indicator_record_multiOD_multiclass_sue_0.01_0613.txt', np.array(indicator_record))
np.savetxt('upper_obj_record_multiOD_multiclass_sue_0.01_0613.txt', np.array(upper_obj_record))
np.savetxt('upper_parameters_record_multiOD_multiclass_sue_0.01_0613.txt', np.array(upper_parameters_record))